## _*Using Qiskit Aqua for exact cover problems*_

In mathematics, given a collection $S$ of subsets of a set $X$.
An exact cover is a subcollection $S_{ec} \subseteq S$ such that each element in $X$ is contained in exactly one subset $\in S_{ec}$. 

We will go through three examples to show (1) how to run the optimization in the non-programming way, (2) how to run the optimization in the programming way, (3) how to run the optimization with the VQE.
We will omit the details for the support of CPLEX, which are explained in other notebooks such as maxcut.

### The problem and the brute-force method.

first, let us take a look at the list of subsets.

In [1]:
import numpy as np
import json
from qiskit import Aer
from qiskit_aqua import run_algorithm
from qiskit_aqua.input import EnergyInput
from qiskit_aqua.translators.ising import exactcover
from qiskit_aqua.algorithms import ExactEigensolver

input_file = 'sample.exactcover'
with open(input_file) as f:
    list_of_subsets = json.load(f)
    print(list_of_subsets)
    qubitOp, offset = exactcover.get_exactcover_qubitops(list_of_subsets)
    algo_input = EnergyInput(qubitOp)

[[2, 3, 4], [1, 2], [3, 4], [1, 2, 3]]


Then we apply the brute-force method. Basically, we exhaustively try all the binary assignments. In each binary assignment, the entry of a subset is either 0 (meaning the subset is not in the cover) or 1 (meaning the subset is in the cover). We print the binary assignment that satisfies the definition of the exact cover.

In [2]:
def brute_force():
    # brute-force way: try every possible assignment!
    has_sol = False

    def bitfield(n, L):
        result = np.binary_repr(n, L)
        return [int(digit) for digit in result]  # [2:] to chop off the "0b" part

    L = len(list_of_subsets)
    max = 2**L
    for i in range(max):
        cur = bitfield(i, L)
        cur_v = exactcover.check_solution_satisfiability(cur, list_of_subsets)
        if cur_v:
            has_sol = True
            break
    return has_sol, cur

has_sol, cur = brute_force()
if has_sol:
    print("solution is", cur)
else:
    print("no solution is found")

solution is [0, 1, 1, 0]


### Part I: run the optimization in the non-programming way

In [3]:
params = {
    'problem': {'name': 'ising'},
    'algorithm': {'name': 'ExactEigensolver'}
}
result = run_algorithm(params, algo_input)
x = exactcover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exactcover.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1, 0])
if exactcover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("solution is", ising_sol)
else:
    print("no solution is found")

solution is [0. 1. 1. 0.]


### Part II: run the optimization in the programming way

In [4]:
algo = ExactEigensolver(algo_input.qubit_op, k=1, aux_operators=[])
result = algo.run()
x = exactcover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exactcover.get_solution(x)
np.testing.assert_array_equal(ising_sol, [0, 1, 1, 0])

if exactcover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("solution is", ising_sol)
else:
    print("no solution is found")

solution is [0. 1. 1. 0.]


### Part III: run the optimization with VQE

In [5]:
algorithm_cfg = {
    'name': 'VQE',
    'operator_mode': 'matrix'
}

optimizer_cfg = {
    'name': 'COBYLA'
}

var_form_cfg = {
    'name': 'RYRZ',
    'depth': 5
}

params = {
    'problem': {'name': 'ising', 'random_seed': 10598},
    'algorithm': algorithm_cfg,
    'optimizer': optimizer_cfg,
    'variational_form': var_form_cfg
}
backend = Aer.get_backend('statevector_simulator')
result = run_algorithm(params, algo_input, backend=backend)
x = exactcover.sample_most_likely(len(list_of_subsets), result['eigvecs'][0])
ising_sol = exactcover.get_solution(x)
if exactcover.check_solution_satisfiability(ising_sol, list_of_subsets):
    print("solution is", ising_sol)
else:
    print("no solution is found")

solution is [0. 1. 1. 0.]
